# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime, time

In [ ]:
# Setting Pandas Options
pd.options.mode.copy_on_write = True
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Data Exploration

In [ ]:
# Reading Data
airlines = pd.read_csv('../sources/data/airlines.csv')
airports = pd.read_csv('../sources/data/airports.csv')
df = pd.read_csv('../sources/data/flights.csv', low_memory=False)
airport_num = pd.read_csv('../sources/data/L_AIRPORT_ID.csv')
airport_ID = pd.read_csv('../sources/data/L_AIRPORT.csv')

In [ ]:
airlines.head()

In [ ]:
airports.head()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Fixing Column Names
df.columns = df.columns.str.title()

In [ ]:
df.columns.tolist()

In [ ]:
df.describe().T

In [ ]:
df.duplicated().sum()

In [ ]:
df.isna().sum() 

# Data Cleaning

## Handling Null Values

In [ ]:
df.isna().sum() * 100 / len(df)

In [ ]:
# Checking Null values when the flight is cancelled
df[df['Cancelled'] == 1].isna().sum() * 100 / len(df[df['Cancelled'] == 1])

In [ ]:
df.dropna(subset=['Scheduled_Time'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.isna().sum() * 100 / len(df)

## Handling Wrong Values

### Categorical Features

In [ ]:
df.select_dtypes(include='object').columns.tolist()

In [ ]:
df['Airline'].value_counts()

In [ ]:
df['Tail_Number'].value_counts()

In [ ]:
df['Tail_Number'].nunique()

In [ ]:
# high variance, dropping
df.drop("Tail_Number", axis=1, inplace=True)

In [ ]:
df['Cancellation_Reason'].value_counts()

In [ ]:
df['Origin_Airport'].nunique()

In [ ]:
df['Origin_Airport'].value_counts()

In [ ]:
df['Destination_Airport'].nunique()

In [ ]:
# Has Numeric values
df['Destination_Airport'].value_counts()

Handling wrong airport names (numeric)

In [ ]:
airports_mapping = pd.merge(airport_num, airport_ID, on='Description')

In [ ]:
airports_mapping.head()

In [ ]:
airports_mapping.shape

In [ ]:
airports_mapping['Code_x'] = airports_mapping['Code_x'].astype(str)

In [ ]:
airports_mapping = airports_mapping.set_index('Code_x')['Code_y'].to_dict()

In [ ]:
airports_mapping

In [ ]:
def map_airports(x):
    if x in airports_mapping.keys():
        return airports_mapping[x]
    else:
        return x

In [ ]:
df['Origin_Airport'] = df['Origin_Airport'].map(map_airports)
df['Destination_Airport'] = df['Destination_Airport'].map(map_airports)

In [ ]:
print(f" Number of Unique Origin Airports: {df['Origin_Airport'].nunique()}")
print(f" Number of Unique Destination Airports: {df['Destination_Airport'].nunique()}")
print(f" Number of Airports : {airports.shape[0]}")

In [ ]:
set(df['Origin_Airport'].unique().tolist()) - set(airports['IATA_CODE'].unique().tolist())

In [ ]:
set(df['Destination_Airport'].unique().tolist()) - set(airports['IATA_CODE'].unique().tolist())

In [ ]:
df[df['Origin_Airport'] == 'BSM'].shape

In [ ]:
df[df['Origin_Airport'] == 'BSM']["Day_Of_Week"].value_counts()

In [ ]:
df.drop(df[(df['Origin_Airport'] == 'BSM') | (df['Destination_Airport'] == 'BSM')].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
del airports_mapping
del airport_num
del airport_ID

### Numerical Features

In [ ]:
df.select_dtypes(include='number').columns.tolist()

In [ ]:
for col in df.select_dtypes(include='number').columns.tolist():
    print(f"{col} : {df[col].nunique()}")
    print(df[col].value_counts().sort_index())
    print(" \n")
    print("#" * 50)

In [ ]:
df.select_dtypes(include='number').columns.tolist()

In [ ]:
df.describe().T

Checking for outliers

In [ ]:
cols_to_check = [   "Departure_Delay", "Taxi_Out", "Scheduled_Time", "Elapsed_Time",
                    "Air_Time", "Distance", "Taxi_In", "Arrival_Delay",
                    "Air_System_Delay", "Security_Delay", "Late_Aircraft_Delay",
                    "Airline_Delay", "Weather_Delay",
                    ] 

In [ ]:
px.histogram(df, x='Departure_Delay')

In [ ]:
df[(df['Departure_Delay'] > 150) | (df['Departure_Delay'] < -50)].shape[0] / len(df)

In [ ]:
df.drop(df[(df['Departure_Delay'] > 150) | (df['Departure_Delay'] < -50)].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
px.histogram(df, x='Taxi_Out')

In [ ]:
df[df['Taxi_Out'] > 50].shape[0] / len(df)

In [ ]:
df.drop(df[df['Taxi_Out'] > 50].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
px.histogram(df, x='Scheduled_Time')

In [ ]:
df[(df['Scheduled_Time'] > 450) | (df['Scheduled_Time'] < 30)].shape[0] / len(df)

In [ ]:
df.drop(df[(df['Scheduled_Time'] > 450) | (df['Scheduled_Time'] < 30)].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
px.histogram(df, x='Elapsed_Time')

In [ ]:
df[(df['Elapsed_Time'] > 450) | (df['Elapsed_Time'] < 25)].shape[0] / len(df)

In [ ]:
df.drop(df[(df['Elapsed_Time'] > 450) | (df['Elapsed_Time'] < 25)].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
px.histogram(df, x='Air_Time')

In [ ]:
px.histogram(df, x='Distance')

In [ ]:
df[(df['Distance'] > 3000) | (df['Distance'] < 100)].shape[0] / len(df)

In [ ]:
df.drop(df[(df['Distance'] > 3000) | (df['Distance'] < 100)].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
px.histogram(df, x='Taxi_In')

In [ ]:
df[(df['Taxi_In'] > 50) | (df['Taxi_In'] < 0)].shape[0] / len(df)

In [ ]:
df.drop(df[(df['Taxi_In'] > 50) | (df['Taxi_In'] < 0)].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
cols_to_check

In [ ]:
px.histogram(df, x='Arrival_Delay')

In [ ]:
df[(df['Arrival_Delay'] > 150) | (df['Arrival_Delay'] < -50)].shape[0] / len(df)

In [ ]:
df.drop(df[(df['Arrival_Delay'] > 150) | (df['Arrival_Delay'] < -50)].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
px.histogram(df, x='Air_System_Delay')

In [ ]:
px.histogram(df, x='Security_Delay')

In [ ]:
px.histogram(df, x='Late_Aircraft_Delay')

In [ ]:
px.histogram(df, x='Airline_Delay')

In [ ]:
px.histogram(df, x='Weather_Delay')

# Feature Engineering

In [ ]:
df.head()

In [ ]:
# Creating Date Column
df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])

In [ ]:
# Function to Format Time
def format_time(value):
        if pd.isna(value):
            return np.nan
        str_value = str(int(value)).zfill(4)  
        hours, minutes = str_value[:2], str_value[2:]
        return hours + ':' + minutes

In [ ]:
# Formatting Time
columns_to_transform = [
    'Scheduled_Departure',
    'Departure_Time',
    'Scheduled_Arrival',
    'Arrival_Time',
    'Wheels_Off',
    'Wheels_On'
]
for column in columns_to_transform:
    df[column] = df[column].apply(format_time)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Checking Correlation
sns.heatmap(df.select_dtypes(exclude='object').corr())

In [ ]:
df.head()

In [ ]:
# Saving Cleaned df
df.to_csv('../sources/data/flights_cleaned.csv', index=False)